In [92]:
import tensorflow as tf
import numpy as np
from keras.layers import Attention, Input, Conv2D, Flatten, Dense, Multiply, Activation, Lambda, Permute, RepeatVector, merge
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

In [47]:
img_batch = np.array([[
                [1,1,1,1],
                [1,5,5,1],
                [1,1,1,1]
                ],
                [
                [1,1,1,1],
                [1,5,5,1],
                [1,1,1,1]
                ]]
            )
img_batch = img_batch.reshape((2,3,4,1))
print(img_batch)

[[[[1]
   [1]
   [1]
   [1]]

  [[1]
   [5]
   [5]
   [1]]

  [[1]
   [1]
   [1]
   [1]]]


 [[[1]
   [1]
   [1]
   [1]]

  [[1]
   [5]
   [5]
   [1]]

  [[1]
   [1]
   [1]
   [1]]]]


In [111]:
inp = Input(shape=(3,4,1))

query = Conv2D(filters=1, kernel_size=1)(inp) #filters = in_dimension//8
key = Conv2D(filters=1, kernel_size=1)(inp) #filters = in_dimension//8
value = Conv2D(filters=1, kernel_size=1)(inp)

out = Attention(use_scale=True)([query, key, value]) #scale adds a learnable parameter applied to the attention scores
out = Activation("sigmoid")(out)
test = Model(inp, out)

test.summary()

Model: "functional_38"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, 3, 4, 1)]    0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 3, 4, 1)      2           input_58[0][0]                   
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 3, 4, 1)      2           input_58[0][0]                   
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 3, 4, 1)      2           input_58[0][0]                   
______________________________________________________________________________________

In [109]:
test(img_batch)

<tf.Tensor: shape=(2, 3, 4, 1), dtype=float32, numpy=
array([[[[0.18451074],
         [0.18451074],
         [0.18451074],
         [0.18451074]],

        [[0.00062261],
         [0.0005926 ],
         [0.0005926 ],
         [0.00062261]],

        [[0.18451074],
         [0.18451074],
         [0.18451074],
         [0.18451074]]],


       [[[0.18451074],
         [0.18451074],
         [0.18451074],
         [0.18451074]],

        [[0.00062261],
         [0.0005926 ],
         [0.0005926 ],
         [0.00062261]],

        [[0.18451074],
         [0.18451074],
         [0.18451074],
         [0.18451074]]]], dtype=float32)>

In [84]:
# checking against manual implementation
input1 = Input(shape=(3,4,1))
e=Dense(1, activation='tanh')(input1)
# Now do all the softmax business taking the above o/p
e=Flatten()(e)
a=Activation('softmax')(e)
temp=RepeatVector(256)(a)
print(temp)
temp=Permute([2,1])(temp)
print(temp)
# multiply weight with lstm layer o/p
output = merge.Multiply()([input1, temp])
# Get the attention adjusted output state
output = Lambda(lambda values: K.sum(values, axis=1))(output)

test2 = Model(input1, output)

Tensor("repeat_vector_28/Tile:0", shape=(None, 256, 12), dtype=float32)
Tensor("permute_26/transpose:0", shape=(None, 12, 256), dtype=float32)


ValueError: Operands could not be broadcast together with shapes (3, 4, 1) (12, 256)